In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from math import isnan
import os

In [ ]:
cid = '91382d101a194c12b67ba7f857bede3e'
secret = '74179f9842234459a51e85c7f65bd180'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
all_playlists_csv = pd.read_csv('/content/drive/MyDrive/CS539 Project/PlaylistCollection.csv')
all_playlists_csv.columns

Index(['Timestamp', 'Name', 'Playlist_link_1', 'Playlist_link_2',
       'Playlist_link_3', 'Playlist_link_4', 'Playlist_link_5', 'Consent'],
      dtype='object')

In [ ]:
df = all_playlists_csv

In [ ]:
'''Convert Dataframe (csv from google form) to a Dictionary of the form: 
{key <- str:person's name, value <- list[playlist links]}'''

dict1 = dict([(i,[x,y,z,a,b]) for i, x,y,z,a,b in zip(df.Name,df.Playlist_link_1, df.Playlist_link_2,df.Playlist_link_3,df.Playlist_link_4, df.Playlist_link_5)])
dict1

{'Oluseun Olulana': [nan,
  'https://open.spotify.com/playlist/37i9dQZF1E37m1zP1rWbTE?si=ST84-1h5SS2zschdWc44Lw',
  nan,
  nan,
  nan],
 'Miheer Diwan ': ['https://open.spotify.com/playlist/1NJZKpVxbsORITxvHbZnZp?si=sh05O33zR_6fATUvfsd1bg&utm_source=copy-link',
  nan,
  nan,
  nan,
  nan],
 'Patrick': ['https://open.spotify.com/playlist/2m231YvffMqvBbkw8RuS2X?si=219b1cbc35c6430d',
  nan,
  nan,
  nan,
  nan],
 'Kartik Nautiyal': ['https://open.spotify.com/playlist/52txfGwWA9qTZJijGuzjbz?si=00fd48cd63474354',
  nan,
  nan,
  nan,
  nan],
 'Ryan Dieselman': ['https://open.spotify.com/playlist/1G6d7DxbK9hPoYNm5DXX5l?si=dZ8nitMOQXKYMM3i6AGsvw',
  'https://open.spotify.com/playlist/3uAKzkAN17tNST5L1tCCFd?si=Ew4Ay6uLRme_doZ5IP10dg',
  'https://open.spotify.com/playlist/3QpSGQsbTmLs9dJHENNch3?si=UBocEMybTByr_xwWMXPdUQ',
  'https://open.spotify.com/playlist/5ubnZsC6wLDmYGFbcIiAo6?si=ttvedrQaSmmvXRsZJrjoBw',
  nan],
 'Chao Wang': ['https://open.spotify.com/playlist/4VSjcDgO8klwJdD7ermEvL?si=1-B

In [ ]:
def cleanNan(dict):
    cleandict = {}
    
    for k, v in dict1.items():
        new = []
        for item in v:
            if type(item) == float:
                if not isnan(item):
                    new.append(item)
            else:
                new.append(item)
        cleandict[k] = new
        
    return cleandict

In [ ]:
dict2 = cleanNan(dict1)
dict2

{'Oluseun Olulana': ['https://open.spotify.com/playlist/37i9dQZF1E37m1zP1rWbTE?si=ST84-1h5SS2zschdWc44Lw'],
 'Miheer Diwan ': ['https://open.spotify.com/playlist/1NJZKpVxbsORITxvHbZnZp?si=sh05O33zR_6fATUvfsd1bg&utm_source=copy-link'],
 'Patrick': ['https://open.spotify.com/playlist/2m231YvffMqvBbkw8RuS2X?si=219b1cbc35c6430d'],
 'Kartik Nautiyal': ['https://open.spotify.com/playlist/52txfGwWA9qTZJijGuzjbz?si=00fd48cd63474354'],
 'Ryan Dieselman': ['https://open.spotify.com/playlist/1G6d7DxbK9hPoYNm5DXX5l?si=dZ8nitMOQXKYMM3i6AGsvw',
  'https://open.spotify.com/playlist/3uAKzkAN17tNST5L1tCCFd?si=Ew4Ay6uLRme_doZ5IP10dg',
  'https://open.spotify.com/playlist/3QpSGQsbTmLs9dJHENNch3?si=UBocEMybTByr_xwWMXPdUQ',
  'https://open.spotify.com/playlist/5ubnZsC6wLDmYGFbcIiAo6?si=ttvedrQaSmmvXRsZJrjoBw'],
 'Chao Wang': ['https://open.spotify.com/playlist/4VSjcDgO8klwJdD7ermEvL?si=1-Bo0MVGS4ie1rfrsF_a8A&pt=354b62410a81a0258a9a39b09d7396aa',
  'https://open.spotify.com/playlist/56u9oePkkALNGdbOgUe2yW?s

In [ ]:
def call_playlist(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    owner = sp.user_playlist(creator,playlist_id)['owner']['display_name']
    playlist_name = sp.user_playlist(creator,playlist_id)['name']
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        try:
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]

            # Get audio features
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_features_list[4:]:
                playlist_features[feature] = audio_features[feature]

            # Concat the dfs
            track_df = pd.DataFrame(playlist_features, index = [0])
            playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
            playlist_df["owner"] = owner
            playlist_df["playlist_name"] = playlist_name
            playlist_df["user"] = creator
        
        except:
            pass
    #Step 3
        
    return playlist_df

In [ ]:
import time

In [ ]:
# record start time
os.chdir('/content/drive/MyDrive/CS539 Project/Playlists 2')

start = time.time()

# Call call_playlist() fuction to get info of each playlist for all users. Save each playlist info as a csv file
for k, v in dict2.items():
    print(k)
    counter = 0

    print(v)
    for item in v:
        print(item)
        counter += 1
        print(counter)
        df = call_playlist(k, item)
        df.to_csv('%s_playlist%i.csv'%(k, counter), index = False)


# record end time
end = time.time()

# print the difference between start and end time in milli. secs
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

Oluseun Olulana
['https://open.spotify.com/playlist/37i9dQZF1E37m1zP1rWbTE?si=ST84-1h5SS2zschdWc44Lw']
https://open.spotify.com/playlist/37i9dQZF1E37m1zP1rWbTE?si=ST84-1h5SS2zschdWc44Lw
1
Miheer Diwan 
['https://open.spotify.com/playlist/1NJZKpVxbsORITxvHbZnZp?si=sh05O33zR_6fATUvfsd1bg&utm_source=copy-link']
https://open.spotify.com/playlist/1NJZKpVxbsORITxvHbZnZp?si=sh05O33zR_6fATUvfsd1bg&utm_source=copy-link
1
Patrick
['https://open.spotify.com/playlist/2m231YvffMqvBbkw8RuS2X?si=219b1cbc35c6430d']
https://open.spotify.com/playlist/2m231YvffMqvBbkw8RuS2X?si=219b1cbc35c6430d
1
Kartik Nautiyal
['https://open.spotify.com/playlist/52txfGwWA9qTZJijGuzjbz?si=00fd48cd63474354']
https://open.spotify.com/playlist/52txfGwWA9qTZJijGuzjbz?si=00fd48cd63474354
1
Ryan Dieselman
['https://open.spotify.com/playlist/1G6d7DxbK9hPoYNm5DXX5l?si=dZ8nitMOQXKYMM3i6AGsvw', 'https://open.spotify.com/playlist/3uAKzkAN17tNST5L1tCCFd?si=Ew4Ay6uLRme_doZ5IP10dg', 'https://open.spotify.com/playlist/3QpSGQsbTmLs9dJHE